# <div style='background:gainsboro; font-family:Georgia; text-align:center'> How to make the ML models easily consumable with FLASK REST API?</div>
***
> ### <div style='font-family:Georgia; text-align: justify'> APIs are essential to make the ML model be accessible; after building ML models, to make use of its results, or to implement it in an application, REST APIs can be used. In this document, I will walk you through the steps from creating a simple ML model to creating, testing a FLASK REST API to make use of the ML model. </div>

![title](Project_Img_title.png)

---
### <div style='text-align: center'> Tools used: VS Code, Anaconda, Jupyter Notebook, Python, FLASK REST API, Numpy, Pandas, and Sci-kit-learn </div>
---
<div style='text-align: center'> Assuming you have installed Anaconda or Miniconda, Python, VS Code on your machine </div>

### <div style='font-family:Georgia'> Begin with creating an Anacond Data Science virtual environment, to be used with Python </div>

#### <div style='font-family:Georgia'>Choose a folder as root directory for this project and navigate to this folder on the Terminal </div>

---
1. Create an Anaconda environment by executing the below command on the Terminal <br>
    conda create -n lifeofpy python=3.7 pandas jupyter seaborn scikit-learn keras tensorflow <br>
    where 'lifeofpy' is a name given to the Anaconda virtual environment <br>
2. Create another folder and choose this as your Workspace on VS Code for this project <br>
3. Open the Workspace in VS Code <br>
4. Open Command Pallet: View > Command Pallette > Python: Select Interpreter <br>
5. Select the Anaconda virtual environment that was created in Step 1 <br>
    In my machine it was 'Python 3.7.7 64-bit ('lifeofpy':conda)' <br>

**The project goal can be achieved without creating a virtual environment. However, creating one is a best practice.**
    
6. By now, VS Code is setup with the required environment to complete the project. Let us now create a Jupyter notebook file in VS Code. Open Command Pallette; View > Command Pallette. Or, press ⇧⌘P > Python: Create New Jupyter Notebook <br>
7. We are now good to work with the dataset and build a simple ML model
---

> The project has the below four important files.
1. Flask_ML_API.ipynb; A Jupyter Notebook file that will have the Python code to process the dataset, ML model and a snippet to respond to the JSON request
2. app.py; A Python script file to handel to create a web server, create API and manage the API calls
3. The other two files are model.pkl and model_columns.pkl files. They are ML object model files saved into byte stream. (Serialization, De-serialization)

**The Project directory structure looks like the image below**
![title](Prj_Dir.png)

#### <div style='text-align:center; color:blue'> The below code goes in Flask_ML_API.ipynb file </div>

#### Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics

#### Get the dataset

In [2]:
from sklearn.datasets import load_boston
bos = load_boston()

### A bit about the dataset    
    The boston house-prices dataset (regression).
    ==============     ==============
    Samples total                 506
    Dimensionality                 13
    Features           real, positive
    Targets             real 5. - 50.
    ==============     ==============
    Returns
    -------
    data : Bunch
     Dictionary-like object, the interesting attributes are:
     'data', the data to learn, 'target', the regression targets, and 'DESCR', the full description of the dataset.

#### Initialize the dataset

In [3]:
bos_init = pd.DataFrame(bos.data)

In [4]:
#Preview of the top five rows
bos_init.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


#### Import feature names from the dataset

In [5]:
bos_init.colums = bos.feature_names
bos_init.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


#### Further need to predict price, create a target variable for the ML model

In [6]:
bos_init['Price'] = bos.target

#### The ML model will need a numerical value rather than categorical value for the predection of price; check if there are any NULL values

In [7]:
bos_init.isnull().sum()

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
Price    0
dtype: int64

#### Check if the values are categorical

In [8]:
bos_init.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       506 non-null    float64
 1   1       506 non-null    float64
 2   2       506 non-null    float64
 3   3       506 non-null    float64
 4   4       506 non-null    float64
 5   5       506 non-null    float64
 6   6       506 non-null    float64
 7   7       506 non-null    float64
 8   8       506 non-null    float64
 9   9       506 non-null    float64
 10  10      506 non-null    float64
 11  11      506 non-null    float64
 12  12      506 non-null    float64
 13  Price   506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


### <div style='font-family:Georgia'> Creating the ML model for the Price prediction <br>
<b> Step 1: Separate features and target variables <br>
Step 2: Split the dataset into training and testing dataset <br>
Step 3: Continue creating the ML model </div>

#### Separate features and target variables

In [9]:
x = bos_init.drop(['Price'], axis = 1)
y = bos_init['Price']

#### Split the dataset into training and testing dataset

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1)
print('Shape of x_train \n', x_train.shape, '\n\n Shape of x_test \n', x_test.shape, '\n-------------\n')
print('Shape of y_train \n', y_train.shape, '\n\n Shape of y_test \n', y_test.shape)

Shape of x_train 
 (404, 13) 

 Shape of x_test 
 (102, 13) 
-------------

Shape of y_train 
 (404,) 

 Shape of y_test 
 (102,)


#### Creating the ML model

In [11]:
from sklearn.ensemble import RandomForestRegressor
classifier = RandomForestRegressor()
classifier.fit(x_train, y_train)

RandomForestRegressor()

#### Model evaluation; evaluate the model performance for training and test dataset

In [12]:
##Training dataset model evaluation
print('Training dataset model evaluation results \n')
prediction = classifier.predict(x_train)
print('R^2: ', metrics.r2_score(y_train, prediction))
print('Mean Abs Error: ', metrics.mean_absolute_error(y_train, prediction))
print('Mean Squared Error: ', metrics.mean_squared_error(y_train, prediction))
print('Root Mean Square Error: ', np.sqrt(metrics.mean_squared_error(y_train, prediction)), '\n')

##Test dataset model evaluation
print('\nTest dataset model evaluation results \n')
prediction_test = classifier.predict(x_test)
print('R^2: ', metrics.r2_score(y_test, prediction_test))
print('Mean Abs Error: ', metrics.mean_absolute_error(y_test, prediction_test))
print('Mean Squared Error: ', metrics.mean_squared_error(y_test, prediction_test))
print('Root Mean Square Error: ', np.sqrt(metrics.mean_squared_error(y_test, prediction_test)))

Training dataset model evaluation results 

R^2:  0.9814199968501339
Mean Abs Error:  0.8210693069306922
Mean Squared Error:  1.500918386138613
Root Mean Square Error:  1.2251197435918715 


Test dataset model evaluation results 

R^2:  0.9155554098202047
Mean Abs Error:  2.2745784313725492
Mean Squared Error:  8.345438401960783
Root Mean Square Error:  2.888847244483651


### To save and use the ML object model files.
#### <div> 1. Create a folder with the name 'model' within the root directory <br> 2. Create two empty files within 'model' folder with the following file name and extension 'model.pkl' and 'model_columns.pkl' </div>
#### Serialization and Deserialization mechanism will be useful to store the ML object model in byte system and the other way round

In [13]:
#Save the model to a file
import pickle
with open('model/model.pkl', 'wb') as file:
    pickle.dump(classifier, file)

#Save the Columns
model_columns = list(x.columns)
with open('model/model_columns.pkl', 'wb') as file:
    pickle.dump(model_columns, file)

---
### <div style='font-family:Georgia; background:GhostWhite; text-align:center'>This marks the end of Flask_ML_API.ipynb file </div>
---

### Let is begin with app.py file to create a web server, API and modules to support the API call

### <div> Install Flask in the Anaconda virtual environment. <br>
1. If the Virtual Environment is not active, execute 'conda activate lifeofpy' before installing Flask <br>
2. pip install Flask <br>
3. Create a Python script file with the name 'app.py' wothin the root directory <br>
4. To test the API with Postman | Download and Install POSTMAN </div>

#### The below code will go in app.py file
```python
# Import the required libraries
    from flask import render_template, request, jsonify, Flask
    import flask
    import traceback
    import pickle
    import pandas as pd

    # Define the App
    app = Flask(__name__)
 
    # Import the models
    with open('model/model.pkl', 'rb') as f:
       classifier = pickle.load (f)
 
    with open('model/model_columns.pkl', 'rb') as f:
       model_columns = pickle.load (f)
 
    @app.route('/')
    def welcome():
       return "Price Prediction for the Boston Housing"
 
    @app.route('/predict', methods=['POST','GET'])
    def predict():
  
       if flask.request.method == 'GET':
           return "Prediction page"
 
       if flask.request.method == 'POST':
           try:
               json_ = request.json
               print(json_)
               query_ = pd.get_dummies(pd.DataFrame(json_))
               query = query_.reindex(columns = model_columns, fill_value = 0)
               prediction = list(classifier.predict(query))
 
               return jsonify({
                   "Predicted house price: ":str(prediction)
               })
 
           except:
               return jsonify({
                   "trace": traceback.format_exc()
                   })
      
    if __name__ == "__main__":
       app.run()
```

---
### <div style='font-family:Georgia; background:GhostWhite; text-align:center'>This marks the end of app.py file</div>
---

## <div style='color:blue'> Sequence of execution <br> <div>
1. Run Flas_ML_API.ipynb file <br>
2. Run app.py <br>
    Upon running app.py, the web server will start at http://localhost:5000/ or http://127.0.0.1:5000/
3. Launch POSTMAN, Change the request type to POST and enter the URL http://localhost:5000/predict or http://127.0.0.1:5000/predict
4. Click on the Body, Change the content type to JSON
5. Enter the below sample JSON query and Click on Send
    
```json
[
    {
        "0":"0.04456",
        "1":"21.0",
        "2":"3.41",
        "3":"5.98",
        "4":"0.745",
        "5":"4.238",
        "6":"46.56",
        "7":"3.709",
        "8":"1.4",
        "9":"300.09",
        "10":"13.65",
        "11":"403.21",
        "12":"5.67"
    }
]
```

### The preview should look like the image below
![title](Json_preview.png)

### The output should look like the image below
![title](API_response.png)

## Notice the Response that shows the predicted Price

### With all the above steps complete, we now have created an FLASK REST API to make use of the ML model.